In [30]:
import whisper
import pandas as pd
import os

In [31]:
RED = '\033[31m'
GREEN_BOLD = '\033[1;32m'
YELLOW = '\033[33m'
BLUE = '\033[34m'
END = '\033[0m'


def print_with_color(text, color):
    print(color + text + END)

In [32]:
def get_lyrics_file_path(folder_name, file_name):
    desktop_path = os.path.expanduser("~/Desktop")
    new_folder_path = os.path.join(desktop_path, folder_name)
    new_folder_path = os.path.join(new_folder_path, "lyrics")
    if not os.path.exists(new_folder_path):
        os.makedirs(new_folder_path)
    file_path = os.path.join(new_folder_path, file_name)

    return file_path

In [33]:
import librosa


def is_valid_transcribe(whisper_df, music_file_path):
    transcribed_len = whisper_df["end"].to_numpy()[-1]
    original_len = librosa.get_duration(path=music_file_path)
    print(transcribed_len, original_len)
    return transcribed_len/original_len > 0.9

In [34]:
def transcribe(path, model_quality, is_forced=False):
    print_with_color(path, BLUE)
    model = whisper.load_model(model_quality)
    target_file_path = get_lyrics_file_path(
        "edited_files", f"{os.path.basename(path).split('.')[0]}.csv")
    if os.path.exists(target_file_path) & (not is_forced):
        print_with_color("already exists", YELLOW)
        return
    result = model.transcribe(
        path, verbose=True, language="ja")
    df_1 = pd.DataFrame(result["segments"])
    is_valid_transcribe(df_1, path)
    df_1.to_csv(target_file_path)

In [35]:
def record_job(dir_path, model_quality, is_forced=False):
    print_with_color("record_job_start", GREEN_BOLD)
    print_with_color(model_quality, BLUE)
    files_path_list = os.listdir(dir_path)
    files_path_list = [os.path.join(dir_path, file_path) for file_path in files_path_list if file_path.endswith(
        ".mp3") or file_path.endswith(".mp4")]
    files_path_list
    for file_path in files_path_list:
        transcribe(file_path, model_quality, is_forced=is_forced)
    print_with_color("record_job done", GREEN_BOLD)

In [ ]:
record_job("/Users/itouhikaru/Downloads", "base", True)